In [5]:
# Analysis Summary:

# During this exercise, we explored several key concepts including dataframe functionality (populating, merging and cleaning), grouping data on 
# specific values (school types and names), and categorizing data within specific ranges (binning). This allowed for presenting of the resultant 
# data in variety # of formats to better explore potential links between various school characteristics and student performance.

# Analysis Outcomes:
# By reviewing the various dataframes created, two conclusions are evident:
# - Dollars spent per student does not result in high numbers of passing students. The number of students that pass both math and reading actually 
# goes down as spending goes up. A similar trend is reflected when looking at school size. The smaller schools perform only slightly better than the
# medium schools, but they both outperform the large schools by a measurable amount (over 30% difference with regards to who passes both math and 
# reading). [Reference spending_summary and school_summary tables.]
# - Charter schools are outperforming District schools in terms of overall passing percentage. Sort on that figure and we see that the top 5 schools 
# are all Charter schools. This would track as the average overall passing percentage gap between the two school types is nearly 40% (90.4% to 53.6%). 


In [6]:
#Module 4 Challenge - Pandas 

#imports
import pandas as pd
from pathlib import Path
import warnings
import numpy as np

#Hide warnings after first review
warnings.filterwarnings(action='once')

In [7]:
#set file paths for data sources
schools = Path("Resources/schools_complete.csv")
students = Path("Resources/students_complete.csv")

In [8]:
#open data files into dataframes
schools_df = pd.read_csv(schools)
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [9]:
#open data files into dataframes
students_df = pd.read_csv(students)
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [10]:
#merge data into one dataframe
district_df = students_df.merge(schools_df, on=["school_name", "school_name"], how='left')
district_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [11]:
#District Summary - Determine data and add to dataframe
ds_series = []
ds_columns = []
#Number of unique schools
district_counts = district_df.groupby('type')['school_name'].nunique()
ds_series.append(district_counts)
ds_columns.append("School Count")
#Number of students
total_students = district_df.drop_duplicates(subset="school_name", keep="first").groupby('type')['size'].sum()
ds_series.append(total_students)
ds_columns.append("Students")
#Total budget
total_budget = district_df.drop_duplicates(subset="school_name", keep="first").groupby('type')['budget'].sum()
ds_series.append(total_budget.map('${:,.2f}'.format))
ds_columns.append("Total Budget")
#Per student budget
student_budget = total_budget.divide(total_students)
ds_series.append(student_budget.map('${:,.2f}'.format))
ds_columns.append("Per Student $")
#Average math score
math_avg = district_df.groupby('type')['math_score'].mean()
ds_series.append(math_avg)
ds_columns.append("Math Avg")
#Average reading score
read_avg = district_df.groupby('type')['reading_score'].mean()
ds_series.append(read_avg)
ds_columns.append("Reading Avg")
#Passing math %
math_pass = district_df[district_df['math_score'] >= 70].groupby('type')['math_score'].count()
ds_series.append(math_pass.divide(total_students).astype(float))
ds_columns.append("Math Pass %")
#Passing reading %
read_pass = district_df[district_df['reading_score'] >= 70].groupby('type')['reading_score'].count()
ds_series.append(read_pass.divide(total_students).astype(float))
ds_columns.append("Reading Pass %")
#total passing 
total_pass = district_df[(district_df['reading_score'] >= 70) & (district_df['math_score'] >= 70)].groupby('type')['student_name'].count()
ds_series.append(total_pass.divide(total_students).astype(float))
ds_columns.append("Pass All %")

district_summary = pd.concat(ds_series, keys=ds_columns, axis=1)
district_summary.head()

,School Count,Students,Total Budget,Per Student $,Math Avg,Reading Avg,Math Pass %,Reading Pass %,Pass All %
type,,,,,,,,,
Charter,8,12194,"$7,301,505.00",$598.78,83.406183,83.902821,0.937018,0.966459,0.905609
District,7,26976,"$17,347,923.00",$643.09,76.987026,80.962485,0.665184,0.809052,0.536959


In [12]:
#School summary - Determine data and add to dateframe
ss_series = []
ss_columns = []

#Set base dataframe:
school_names = district_df.groupby('school_name')['type'].first()
ss_series.append(school_names)
ss_columns.append("School Type")
#Number of students
total_students = district_df.groupby('school_name')['student_name'].count()
ss_series.append(total_students)
ss_columns.append("Students")
#Total budget
total_budget = district_df.drop_duplicates(subset="school_name", keep="first").groupby('school_name')['budget'].sum()
ss_series.append(total_budget)
ss_columns.append("Total Budget")
#Per student budget
student_budget = total_budget.divide(total_students)
ss_series.append(student_budget)
ss_columns.append("Spend Per Student")
#Average math score
math_avg = district_df.groupby('school_name')['math_score'].mean()
ss_series.append(math_avg)
ss_columns.append("Math Avg")
#Average reading score
read_avg = district_df.groupby('school_name')['reading_score'].mean()
ss_series.append(read_avg)
ss_columns.append("Reading Avg")
#Passing math %
math_pass = district_df[district_df['math_score'] >= 70].groupby('school_name')['math_score'].count()
ss_series.append(math_pass.divide(total_students).astype(float))
ss_columns.append("Math Pass %")
#Passing reading %
read_pass = district_df[district_df['reading_score'] >= 70].groupby('school_name')['reading_score'].count()
ss_series.append(read_pass.divide(total_students).astype(float))
ss_columns.append("Reading Pass %")
#total passing 
total_pass = district_df[(district_df['reading_score'] >= 70) & (district_df['math_score'] >= 70)].groupby('school_name')['student_name'].count()
ss_series.append(total_pass.divide(total_students).astype(float))
ss_columns.append("Pass All %")

school_summary = pd.concat(ss_series, keys=ss_columns, axis=1)
school_summary.head()

,School Type,Students,Total Budget,Spend Per Student,Math Avg,Reading Avg,Math Pass %,Reading Pass %,Pass All %
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.542899
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995


In [13]:
#Top Schools - sort schools by overall passing % descending and show top 5
top_schools = school_summary.sort_values(by=['Pass All %'], ascending=False)
top_schools.head()

,School Type,Students,Total Budget,Spend Per Student,Math Avg,Reading Avg,Math Pass %,Reading Pass %,Pass All %
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


In [14]:
#Lowest Schools - sort schools ascending and show top 5
bottom_schools = school_summary.sort_values(by=['Pass All %'], ascending=True)
bottom_schools.head()

,School Type,Students,Total Budget,Spend Per Student,Math Avg,Reading Avg,Math Pass %,Reading Pass %,Pass All %
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


In [15]:
# Use the code provided to separate the data by grade
ninth_graders = district_df[(district_df["grade"] == "9th")]
tenth_graders = district_df[(district_df["grade"] == "10th")]
eleventh_graders = district_df[(district_df["grade"] == "11th")]
twelfth_graders = district_df[(district_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.concat(
    [ninth_grade_math_scores, tenth_grader_math_scores,
     eleventh_grader_math_scores, twelfth_grader_math_scores], 
    keys=["9th", "10th", "11th", "12th"], 
    axis=1
    )

# Minor data wrangling
#math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


In [16]:
# Group by `school_name` and take the mean of the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
reading_scores_by_grade = pd.concat(
    [ninth_grade_reading_scores, tenth_grader_reading_scores,
     eleventh_grader_reading_scores, twelfth_grader_reading_scores], 
    keys=["9th", "10th", "11th", "12th"], 
    axis=1
    )

# Minor data wrangling
#math_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


In [17]:
#Scores by school spending
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_summary['Spending Ranges Per Student'] = pd.cut(
    np.array(school_summary['Spend Per Student'].astype(int)), spending_bins, labels=labels
    )

school_summary.head()

,School Type,Students,Total Budget,Spend Per Student,Math Avg,Reading Avg,Math Pass %,Reading Pass %,Pass All %,Spending Ranges Per Student
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423,$585-630
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348,<$585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045,$630-645
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.542899,$630-645
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995,$585-630


In [18]:
#Calc spending by averages and add to dataframe
spending_series = []
spending_name = []

#spending and math avg
spending_math_scores = school_summary.groupby(["Spending Ranges Per Student"])["Math Avg"].mean()
spending_series.append(spending_math_scores)
spending_name.append("Avg Math Score")
#spending and reading avg
spending_reading_scores = school_summary.groupby(["Spending Ranges Per Student"])["Reading Avg"].mean()
spending_series.append(spending_reading_scores)
spending_name.append("Avg Reading Score")
#spending and passing math avg
spending_passing_math = school_summary.groupby(["Spending Ranges Per Student"])["Math Pass %"].mean()
spending_series.append(spending_passing_math)
spending_name.append("Avg Math Pass %")
#spending and passing reading avg
spending_passing_reading = school_summary.groupby(["Spending Ranges Per Student"])["Reading Pass %"].mean()
spending_series.append(spending_passing_reading)
spending_name.append("Avg Reading Pass %")
#spending and pass all avg
overall_passing_spending = school_summary.groupby(["Spending Ranges Per Student"])["Pass All %"].mean()
spending_series.append(overall_passing_spending)
spending_name.append("Avg Passing All %")

spending_summary = pd.concat(spending_series, keys=spending_name, axis=1)
spending_summary.head()

C:\Users\spend\AppData\Local\Temp\ipykernel_2064\616439614.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_summary.groupby(["Spending Ranges Per Student"])["Math Avg"].mean()


,Avg Math Score,Avg Reading Score,Avg Math Pass %,Avg Reading Pass %,Avg Passing All %
Spending Ranges Per Student,,,,,
<$585,83.455399,83.933814,0.934601,0.966109,0.903695
$585-630,81.899826,83.155286,0.871335,0.927182,0.814186
$630-645,78.518855,81.624473,0.734842,0.843918,0.628577
$645-680,76.997210,81.027843,0.661648,0.811340,0.535269


In [19]:
#Scores by school size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary['School Size'] = pd.cut(
    np.array(school_summary['Students']), size_bins, labels=labels
    )

#Calc size averages and add to dataframe
size_series = []
size_name = []

size_math_scores = school_summary.groupby(["School Size"])["Math Avg"].mean()
size_series.append(size_math_scores)
size_name.append("Math Avg")
size_reading_scores = school_summary.groupby(["School Size"])["Reading Avg"].mean()
size_series.append(size_reading_scores)
size_name.append("Reading Avg")
size_passing_math = school_summary.groupby(["School Size"])["Math Pass %"].mean()
size_series.append(size_passing_math)
size_name.append("Math Pass %")
size_passing_reading = school_summary.groupby(["School Size"])["Reading Pass %"].mean()
size_series.append(size_passing_reading)
size_name.append("Reading Pass %")
size_overall_passing = school_summary.groupby(["School Size"])["Pass All %"].mean()
size_series.append(size_overall_passing)
size_name.append("Overall Pass %")

size_summary = pd.concat(size_series, keys=size_name, axis=1)
size_summary.head()

C:\Users\spend\AppData\Local\Temp\ipykernel_2064\1529727647.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = school_summary.groupby(["School Size"])["Math Avg"].mean()


,Math Avg,Reading Avg,Math Pass %,Reading Pass %,Overall Pass %
School Size,,,,,
Small (<1000),83.821598,83.929843,0.935502,0.960994,0.898839
Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.906215
Large (2000-5000),77.746417,81.344493,0.699634,0.827666,0.582860


In [20]:
#Scores by school type
type_series = []
type_name = []

# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = school_summary.groupby(["School Type"])["Math Avg"].mean()
type_series.append(average_math_score_by_type)
type_name.append("Math Avg")
average_reading_score_by_type = school_summary.groupby(["School Type"])["Reading Avg"].mean()
type_series.append(average_reading_score_by_type)
type_name.append("Reading Avg")
average_percent_passing_math_by_type = school_summary.groupby(["School Type"])["Math Pass %"].mean()
type_series.append(average_percent_passing_math_by_type)
type_name.append("Math Pass %")
average_percent_passing_reading_by_type = school_summary.groupby(["School Type"])["Reading Pass %"].mean()
type_series.append(average_percent_passing_reading_by_type)
type_name.append("Reading Pass %")
average_percent_overall_passing_by_type = school_summary.groupby(["School Type"])["Pass All %"].mean()
type_series.append(average_percent_overall_passing_by_type)
type_name.append("Overall Pass %")

type_summary = pd.concat(type_series, keys=type_name, axis=1)
type_summary.head()

,Math Avg,Reading Avg,Math Pass %,Reading Pass %,Overall Pass %
School Type,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.904322
District,76.956733,80.966636,0.665485,0.807991,0.536722
